In [1]:
import numpy as np
import scipy.stats as stats
from math import log, sqrt, exp
from scipy.stats import norm
import sys
sys.path.append('/Users/ansel_li/Fintech545/public/')
from risk_mgmt import black_s
from scipy.optimize import minimize
from datetime import datetime

In [2]:
# Input parameters
S = 151.03  # Current stock price
K = 165  # Strike price
t = (np.datetime64('2022-03-13') - np.datetime64('2022-03-13')).astype(int) / 365.0
T = (np.datetime64('2022-04-15') - np.datetime64('2022-03-13')).astype(int) / 365.0
r = 0.0425  # Risk-free interest rate
q = 0.0053  # Continuously compounding dividend yield
sigma = 0.20  # Implied volatility


In [3]:
def gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call'):
    tau = T - t
    d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
    d2 = d1 - sigma * sqrt(tau)

    N_d1 = stats.norm.cdf(d1)
    N_d2 = stats.norm.cdf(d2)

    if option_type == 'call':
        delta = exp(-q * tau) * N_d1
        gamma = exp(-q * tau) * N_d1 / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * N_d1 * sigma / (2 * np.sqrt(tau)) - r * K * np.exp(-r * tau) * -N_d2 + q * S * np.exp(-q * tau) * -N_d1
        rho = K * tau * np.exp(-r * tau) * norm.cdf(d2)
    elif option_type == 'put':
        delta = -exp(-q * tau) * (1 - N_d1)
        gamma = exp(-q * tau) * N_d1 / (S * sigma * sqrt(tau))
        vega = S * np.exp(-q * tau) * norm.pdf(d1) * np.sqrt(tau)
        theta = -S * np.exp(q * tau) * N_d1 * sigma / (2 * np.sqrt(tau)) + r * K * np.exp(-r * tau) * -N_d2 - q * S * np.exp(-q * tau) * -N_d1
        rho = -K * tau * np.exp(-r * tau) * norm.cdf(-d2)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    return delta, gamma, vega, theta, rho

In [4]:
call_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='call')
put_greeks = gbsm_greeks(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks)
print("Put Greeks: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks)

Call Greeks: Delta, Gamma, Vega, Theta, Rho
(0.08297130333914773, 0.009135328237370505, 6.938710929513443, -3.7196562346632, 1.1025939156368187)
Put Greeks: Delta, Gamma, Vega, Theta, Rho
(-0.9165496333661425, 0.009135328237370505, 6.938710929513443, -4.623431322046896, -13.758003122735788)


In [5]:
def finite_difference(S, K, t, T, r, q, sigma, option_type='call', delta_shift=0.01):
    S_up = S + S * delta_shift
    S_down = S - S * delta_shift
    sigma_up = sigma + delta_shift
    sigma_down = sigma - delta_shift
    r_up = r + delta_shift
    r_down = r - delta_shift

    def option_price(S, K, t, T, r, q, sigma, option_type):
        tau = T - t
        d1 = (log(S / K) + (r - q + 0.5 * sigma**2) * tau) / (sigma * sqrt(tau))
        d2 = d1 - sigma * sqrt(tau)

        if option_type == 'call':
            return S * exp(-q * tau) * stats.norm.cdf(d1) - K * exp(-r * tau) * stats.norm.cdf(d2)
        elif option_type == 'put':
            return K * exp(-r * tau) * stats.norm.cdf(-d2) - S * exp(-q * tau) * stats.norm.cdf(-d1)

    # Finite difference calculations
    delta_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - option_price(S_down, K, t, T, r, q, sigma, option_type)) / (2 * S * delta_shift)
    gamma_fd = (option_price(S_up, K, t, T, r, q, sigma, option_type) - 2 * option_price(S, K, t, T, r, q, sigma, option_type) + option_price(S_down, K, t, T, r, q, sigma, option_type)) / (S * delta_shift)**2
    vega_fd = (option_price(S, K, t, T, r, q, sigma_up, option_type) - option_price(S, K, t, T, r, q, sigma_down, option_type)) / (2 * 0.01)
    theta_fd = - (option_price(S, K, t + 1/365, T, r, q, sigma, option_type) - option_price(S, K, t, T, r, q, sigma, option_type)) / (-1/365)
    rho_fd = (option_price(S, K, t, T, r_up, q, sigma, option_type) - option_price(S, K, t, T, r_down, q, sigma, option_type)) / (2 * 0.01)

    return delta_fd, gamma_fd, vega_fd, theta_fd, rho_fd

In [6]:
call_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='call')
put_greeks_fd = finite_difference(S, K, t, T, r, q, sigma, option_type='put')

print("Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(call_greeks_fd)
print("Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho")
print(put_greeks_fd)

import pandas as pd

# Combine the results in a dictionary
greek_data = {
    "Call Closed-form": call_greeks,
    "Call Finite Diff.": call_greeks_fd,
    "Put Closed-form": put_greeks,
    "Put Finite Diff.": put_greeks_fd
}

# Create a DataFrame
greeks_comparison = pd.DataFrame(greek_data, index=["Delta", "Gamma", "Vega", "Theta", "Rho"])

# Display the DataFrame
print(greeks_comparison)

Call Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(0.08390256513620564, 0.016848978828828364, 6.932942300084921, -8.048047189743537, 1.1026982009081365)
Put Greeks Finite Difference: Delta, Gamma, Vega, Theta, Rho
(-0.9156183715690898, 0.016848978828841604, 6.9329423000851875, -1.8621154051211875, -13.757900862010786)
       Call Closed-form  Call Finite Diff.  Put Closed-form  Put Finite Diff.
Delta          0.082971           0.083903        -0.916550         -0.915618
Gamma          0.009135           0.016849         0.009135          0.016849
Vega           6.938711           6.932942         6.938711          6.932942
Theta         -3.719656          -8.048047        -4.623431         -1.862115
Rho            1.102594           1.102698       -13.758003        -13.757901


In [7]:
def binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r) * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Create the stock price tree
    stock = np.zeros((n + 1, n + 1))
    stock[0, 0] = S

    for i in range(1, n + 1):
        stock[i, 0] = stock[i - 1, 0] * u
        for j in range(1, i + 1):
            stock[i, j] = stock[i - 1, j - 1] * d

    # Adjust stock prices for dividends
    if div_date is not None and exercise_type == 'american':
        div_steps = int((div_date - np.datetime64('2022-03-13')).astype(int) / 365.0 * n)
        stock[div_steps:] -= div_amt

    # Create the option price tree
    option = np.zeros_like(stock)

    # Calculate the option's value at maturity
    if option_type == 'call':
        option[:, -1] = np.maximum(stock[:, -1] - K, 0)
    elif option_type == 'put':
        option[:, -1] = np.maximum(K - stock[:, -1], 0)

    # Step back through the tree to calculate the option's value at each node
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option[i, j] = discount * (p * option[i + 1, j] + (1 - p) * option[i + 1, j + 1])

            # Calculate the option's value if exercised early
            if exercise_type == 'american':
                if option_type == 'call':
                    early_exercise = np.maximum(stock[i, j] - K, 0)
                elif option_type == 'put':
                    early_exercise = np.maximum(K - stock[i, j], 0)

                option[i, j] = np.maximum(option[i, j], early_exercise)

    return option




def binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=None, div_amt=0, delta_shift=0.01, rate_shift=0.0001, vol_shift=0.01):
    option_tree = binomial_tree(S, K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_up = binomial_tree(S * (1 + delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_S_down = binomial_tree(S * (1 - delta_shift), K, T, r, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_up = binomial_tree(S, K, T, r, sigma + vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_vol_down = binomial_tree(S, K, T, r, sigma - vol_shift, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_up = binomial_tree(S, K, T, r + rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)
    option_tree_rate_down = binomial_tree(S, K, T, r - rate_shift, sigma, n, option_type, exercise_type, div_date, div_amt)

    # Calculate Greeks
    delta = (option_tree_S_up[0, 0] - option_tree_S_down[0, 0]) / (2 * S * delta_shift)
    gamma = (option_tree_S_up[0, 0] - 2 * option_tree[0, 0] + option_tree_S_down[0, 0])/ (S * delta_shift) ** 2
    theta = -(option_tree[0, 0] - binomial_tree(S, K, T - 1/n, r, sigma, n, option_type, exercise_type, div_date, div_amt)[0, 0]) / (1/n)

    vega = (option_tree_vol_up[0, 0] - option_tree_vol_down[0, 0]) / (2 * vol_shift)
    rho = (option_tree_rate_up[0, 0] - option_tree_rate_down[0, 0]) / (2 * rate_shift)

    return delta, gamma, theta, vega, rho



In [8]:
n=100
div_date = np.datetime64('2022-04-11')
div_amt = 0.88  # Dividend amount

call_without_div = binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american')[0, 0]
put_without_div = binomial_tree(S, K, T, r, sigma, n, option_type='put', exercise_type='american')[0, 0]
call_with_div = binomial_tree(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=div_amt)[0, 0]
put_with_div = binomial_tree(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=div_amt)[0, 0]

print("American Call Option without Dividends:", call_without_div)
print("American Put Option without Dividends:", put_without_div)
print("American Call Option with Dividends:", call_with_div)
print("American Put Option with Dividends:", put_with_div)

American Call Option without Dividends: 0.3317910184978169
American Put Option without Dividends: 14.016650966102587
American Call Option with Dividends: 0.27365120446834945
American Put Option with Dividends: 14.86187574483816


In [9]:
# calculate greeks
call_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american')
put_greeks_without_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american')
call_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='call', exercise_type='american', div_date=div_date, div_amt=div_amt)
put_greeks_with_div = binomial_tree_greeks(S, K, T, r, sigma, n, option_type='put', exercise_type='american', div_date=div_date, div_amt=div_amt)

# create dataframe for greeks
greek_labels = ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']
call_greeks = pd.DataFrame({'Call without div': call_greeks_without_div, 'Call with div': call_greeks_with_div}, index=greek_labels)
put_greeks = pd.DataFrame({'Put without div': put_greeks_without_div, 'Put with div': put_greeks_with_div}, index=greek_labels)

# display greeks dataframes
print('Call Option Greeks:')
print(call_greeks)

print('\nPut Option Greeks:')
print(put_greeks)

Call Option Greeks:
       Call without div  Call with div
Delta          0.082067       0.072438
Gamma          0.016219       0.014888
Theta         -7.663421      -6.868668
Vega           7.031265       5.894682
Rho            1.080959       0.917994

Put Option Greeks:
       Put without div  Put with div
Delta        -0.957268     -0.964653
Gamma         0.020478      0.014834
Theta        -2.258358     -1.720999
Vega          3.751046      2.939957
Rho          -2.847535     -2.823484


In [10]:
# Calculate the sensitivity of the call and put to a change in dividend amount
Delta_call_dividend = (call_with_div - call_without_div) / div_amt
Delta_put_dividend = (put_with_div - put_without_div) / div_amt

print("Delta Call with Dividend: ", Delta_call_dividend)
print("Delta Put with Dividend: ", Delta_put_dividend)

Delta Call with Dividend:  -0.06606797048803119
Delta Put with Dividend:  0.9604827031086066


# Problem 2

In [11]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from risk_mgmt import simulation, VaR, calc_return
# Load the data from CSV files
portfolio = pd.read_csv('problem2.csv')
prices = pd.read_csv('DailyPrices.csv')



# Set the date and other parameters
dividend = 1
rf = 0.0425
q = 0.0053
current_date = '2023-03-03'
current_date = pd.to_datetime(current_date)
S = 151.03


In [12]:
# call the function that add implied vol and T to the portfolio.csv
portfolio = black_s.add_VolnT(S,q,rf, portfolio, current_date)
portfolio.head()

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,T,Implied_Volatility
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.267740
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.134247,0.259735
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.267740
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.134247,0.259735
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,0.134247,0.267740


In [13]:
# Filter the historical prices for AAPL
return_aapl = simulation.log_returns(prices['AAPL'])
m_return = np.mean(return_aapl)
return_aapl = return_aapl - m_return
mu, std = norm.fit(return_aapl)
n_simulations = 10000
n_days = 10
simulated_returns = np.random.normal(loc=mu, scale=std, size=(n_simulations, n_days))


In [14]:
initial_price = 151.03

# Calculate the price series
price_series = pd.DataFrame(initial_price * np.exp(simulated_returns.cumsum(axis=1)))
simulation_price = price_series.iloc[:,9]

In [15]:
simulated_returns[:,-1]

array([-0.01511339,  0.01422379, -0.0286699 , ..., -0.03543034,
       -0.00091159,  0.01921322])

In [16]:
# Get a list of unique portfolio names
portfolios = portfolio['Portfolio'].unique()

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=portfolios, index=['Mean', 'VaR', 'ES'])

# Iterate through each portfolio and calculate the Mean, VaR, and ES
for portname in portfolios:
    portfolio_data = portfolio[portfolio['Portfolio'] == portname]
    simulate_portfolio_values = pd.DataFrame(black_s.portfolio_value(portfolio_data, simulation_price, rf, q, current_date, day_ahead=10))
    current_portfolio_values = pd.DataFrame(black_s.portfolio_value(portfolio_data, [initial_price], rf, q, current_date))

    # Calculate the difference between simulated and current portfolio values
    current_portfolio_value = current_portfolio_values.values[0][0]
    diff_portfolio_values = simulate_portfolio_values.apply(lambda x: x - current_portfolio_value)

    #print(diff_portfolio_values.values[0])
    # Calculate the Mean, VaR, and ES for the difference
    Mean = diff_portfolio_values.mean().values[0]

    var = VaR.calculate_var(diff_portfolio_values.values)
    es = VaR.calculate_es(diff_portfolio_values.values)

    # Add the results to the DataFrame
    results_df.at['Mean', portname] = Mean
    results_df.at['VaR', portname] = var
    results_df.at['ES', portname] = es

# Display the results
results_df.T





,Mean,VaR,ES
Straddle,1.8266,1.377529,1.386567
SynLong,0.182951,16.937507,20.376285
CallSpread,-0.053932,3.967238,4.216356
PutSpread,0.302207,2.690571,2.842846
Stock,0.389033,16.690125,20.109104
Call,1.004776,6.125591,6.401542
Put,0.821824,4.446537,4.641977
CoveredCall,-0.732194,12.840384,16.165104
ProtectedPut,1.093439,8.228482,8.754811


In [17]:
def delta(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

    if option_type == 'Call':
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'Put':
        delta = np.exp(-q * T) * (norm.cdf(d1) - 1)

    return delta

def delta_normal_var_pnl(portfolio, initial_price, rf, q, current_date, simulated_prices):
    portfolio_delta = 0

    for _, row in portfolio.iterrows():
        holding = row['Holding']

        if row['Type'] == 'Stock':
            portfolio_delta += holding
        elif row['Type'] == 'Option':
            T = ((row['ExpirationDate'] - current_date) / pd.Timedelta(1, 'D')) / 365
            sigma = row['Implied_Volatility']
            option_delta = delta(initial_price, row['Strike'], T, rf, q, sigma, row['OptionType'])
            portfolio_delta += holding * option_delta

    simulated_pnl = (simulated_prices - initial_price) * portfolio_delta
    dn_var = np.percentile(simulated_pnl, 5)
    return dn_var, simulated_pnl


results_df1 = pd.DataFrame(columns=portfolios, index=['VaR', 'ES'])
# Calculate Delta-Normal VaR and ES for each portfolio
for portname in portfolios:
    portfolio_data = portfolio[portfolio['Portfolio'] == portname]
    dn_var, pnl = delta_normal_var_pnl(portfolio_data, initial_price, rf, q, current_date, simulation_price)
    dn_es = np.mean(pnl[pnl <= dn_var])

    results_df1.at['VaR', portname] = -dn_var
    results_df1.at['ES', portname] = -dn_es
results_df1.T

,VaR,ES
Straddle,2.262427,2.725887
SynLong,16.663485,20.077006
CallSpread,4.711322,5.676438
PutSpread,4.267388,5.509848
Stock,16.690125,20.109104
Call,9.462956,11.401447
Put,7.999304,10.32832
CoveredCall,9.584515,11.547907
ProtectedPut,11.635643,14.01921


In [27]:
import pandas as pd
import statsmodels.api as sm

# Load daily price data
prices_df = pd.read_csv('DailyPrices.csv', index_col=0, parse_dates=True)

# Load Fama French 3 factor return time series data
ff3_df = pd.read_csv('F-F_Research_Data_Factors_daily.csv', index_col=0, parse_dates=True)
ff3_df.index.name = 'Date'
ff3_df = ff3_df[['Mkt-RF', 'SMB', 'HML', 'RF']]

# Load Carhart Momentum time series data
mom_df = pd.read_csv('F-F_Momentum_Factor_daily.csv', index_col=0, parse_dates=True)
mom_df = mom_df[['Mom']]

# Merge daily prices, Fama French 3 factor and Carhart Momentum data
df = prices_df.join(ff3_df, how='inner').join(mom_df, how='inner')

# Check the DataFrame after joining
print(df.head())
# Define the stocks we want to fit the model for
stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']

# Create an empty DataFrame to store results
results_df = pd.DataFrame(columns=['Stock', 'alpha', 'beta_mkt', 'beta_smb', 'beta_hml', 'beta_mom'])

# Loop through the stocks and fit the 4 factor model for each
for stock in stocks:
    # Define the dependent and independent variables
    y = df[stock] - df['RF']
    
    X = sm.add_constant(df[['Mkt-RF', 'SMB', 'HML', 'Mom']])
    # Fit the 4 factor model
    model = sm.OLS(y, X)
    results = model.fit()
    # Store the results in the results DataFrame
    results_df.loc[len(results_df)] = [stock, results.params['const'], results.params['Mkt-RF'], results.params['SMB'], results.params['HML'], results.params['Mom']]

# Print the results
print(results_df)


                   SPY        AAPL        MSFT        AMZN        TSLA  \
Date                                                                     
2022-02-14  432.011322  167.863144  292.261475  155.167007  291.920013   
2022-02-15  438.978333  171.749573  297.680664  156.510498  307.476654   
2022-02-16  439.470337  171.511032  297.333191  158.100494  307.796661   
2022-02-17  430.082642  167.863144  288.626679  154.652496  292.116669   
2022-02-18  427.297852  166.292648  285.846893  152.601502  285.660004   

                 GOOGL        GOOG        META        NVDA       BRK-B  ...  \
Date                                                                    ...   
2022-02-14  135.526001  135.300003  217.699997  242.443298  314.279999  ...   
2022-02-15  136.608505  136.425507  221.000000  264.702484  316.200012  ...   
2022-02-16  137.738007  137.487503  216.539993  264.862305  315.649994  ...   
2022-02-17  132.539002  132.308502  207.710007  244.841064  313.549988  ...   
2022-02

In [41]:
# Read all data
ff3 = pd.read_csv("F-F_Research_Data_Factors_daily.CSV")
mom = pd.read_csv("F-F_Momentum_Factor_daily.CSV")
# prices = pd.read_csv("DailyPrices.csv")
# returns = calc_return.return_calculate(prices, date_column="Date")
rf = 0.0025
returns = pd.read_csv('DailyReturn.csv')
# Join the FF3 data with the Momentum Data
ffData = pd.merge(ff3, mom, on="Date")
ffData.rename(columns={ffData.columns[-1]: "Mom"}, inplace=True)
ffData.iloc[:, 1:] = ffData.iloc[:, 1:].div(100)
ffData['Date'] = pd.to_datetime(ffData['Date'].astype(str), format="%Y%m%d")

returns['Date'] = pd.to_datetime(returns['Date'], format="%m/%d/%Y")
# Our 20 stocks
stocks = ['AAPL', 'META', 'UNH', 'MA', 'MSFT', 'NVDA', 'HD', 'PFE', 'AMZN', 'BRK-B', 'PG', 'XOM', 'TSLA', 'JPM', 'V', 'DIS', 'GOOGL', 'JNJ', 'BAC', 'CSCO']
stocks = ['AAPL', 'MSFT', 'BRK-B', 'CSCO', 'JNJ']
# Data set of all stock returns and FF3+1 returns
to_reg = pd.merge(returns[['Date'] + stocks], ffData, on="Date")

xnames = ["Mkt-RF", "SMB", "HML", "Mom"]

# OLS Regression for all Stocks
X = np.column_stack((np.ones(len(to_reg)), to_reg[xnames].values))
Y = to_reg[stocks].values

Betas = np.linalg.inv(X.T @ X) @ X.T @ Y
Betas = Betas.T

# Calculate the means of the last 10 years of factor returns
# adding the 0.0 at the front to 0 out the fitted alpha in the next step
means = np.concatenate(([0.0], ffData.loc[ffData['Date'] >= datetime(2013, 2, 9), xnames].mean()))

# Discrete Returns, convert to Log Returns and scale to 1 year
stockMeans = np.log(1 + Betas @ means) * 255 + rf
covar = np.cov(np.log(1.0 + Y.T)) * 255


In [34]:
def max_sharpe_ratio_weights(cov_m, exp_returns, rf):
    num_stocks = len(exp_returns)
    
    # Define the Sharpe Ratio objective function to be minimized
    def neg_sharpe_ratio(weights):
        port_return = np.dot(weights, exp_returns)
        port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_m, weights)))
        sharpe_ratio = (port_return - rf) / port_volatility
        return -sharpe_ratio
    
    # Define the constraints
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # The sum of the weights must be 1
    bounds = tuple([(0, 1) for i in range(num_stocks)]) # The weights must be between 0 and 1
    
    # Find the portfolio weights that maximize the Sharpe Ratio
    initial_weights = np.ones(num_stocks) / num_stocks # Start with equal weights
    opt_results = minimize(neg_sharpe_ratio, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_results.x.round(4), opt_results.fun 


In [42]:
weights, sharpe = max_sharpe_ratio_weights(covar, stockMeans, 0.0425)
print("Sharpe ratio is:", -sharpe)

Sharpe ratio is: 0.5079904796335004


In [43]:
portfolio_weights = np.array(weights)
portfolio_weights

array([0.0842, 0.3112, 0.5039, 0.1008, 0.    ])

In [ ]:
alphas.head()

,Alpha
AAPL,-0.006524
META,-0.006976
UNH,-0.005606
MA,-0.005924
MSFT,-0.006178


In [32]:
expected_return = portfolio_weights.dot(stockMeans)
print(expected_return)

0.15433742718331794
